In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from simple_function import *
import scanpy as sc
import os
import gzip
from bedfile_reader import ReadBED
import pyranges as pr
from scipy import stats

In [2]:

#chosen_region => signal column
#regions_start = list of region starting sites

In [3]:
#loading merged regions
h3k27ac_gene_merged = read_feather_hm("../processing_data/H3K27ac/H3K27ac_peakcount_merged_gene.feather")
h3k27ac_reg_merged = read_feather_hm("../processing_data/H3K27ac/H3K27ac_peakcount_merged_reg.feather")
h3k27ac_non_gene_merged = read_feather_hm("../processing_data/H3K27ac/H3K27ac_peakcount_merged_none_gene.feather")

In [4]:
print(f"gene: {h3k27ac_gene_merged.shape}")
print(f"reg: {h3k27ac_reg_merged.shape}")
print(f"non_gene: {h3k27ac_non_gene_merged.shape}")

gene: (423, 69558)
reg: (423, 22940)
non_gene: (423, 12)


In [ ]:
#df_to_BED("../regions_BED/H3K27ac_gene_domain_expanding.bed", h3k27ac_gene_domains)
#df_to_BED("../regions_BED/H3K27ac_reg_domain_expanding.bed", h3k27ac_reg_domains)
#df_to_BED("../regions_BED/H3K27ac_non_gene_domain_expanding.bed", h3k27ac_non_gene_domains)

## set up 


### get signalvalue by reading files and extract peaks

In [ ]:
def average_signalValue(data: pd.DataFrame, bedfiles_path: str) -> pd.DataFrame:
    #data: pd.DataFrame with region and peak count from each sample, sorted columns accrodding to their position of chr1
    #bedfiles_path: path to the sorted bedfile of sample
    
    #creating signalValue dataframe
    signal_df = pd.DataFrame(0, index = data.index, columns = data.columns)
    
    # get starting site of each columns
    chosen_regions = list(signal_df.columns)
    regions = []
    start_sites = []
    for region in chosen_regions:
        _, pos = region.split(':')
        s, e = pos.split('-')
        start_sites.append(s)
        regions.append([int(s), int(e)])

    #np.array of start sites
    start_sites = np.array(start_sites).astype('int')
    
    #file list in path
    hm_files = os.listdir(bedfiles_path)
    
    # getting signalValue
    # sorted file -> chr1 appears first 
    for f in hm_files:
        name = f.split('_')[1].split('.bed')[0]
        signalValue = []
        with gzip.open(f'{bedfiles_path}/{f}', 'rb') as peaks:
            lines = peaks.readlines()
            for line in lines:
                line = line.decode('utf-8')
                if line.startswith('chr1\t'):
                    start, end, signalValue, peak_pos = [line.split('\t')[i] for i in [1, 2, 6, 9]]
                    peak = int(start) + int(peak_pos)
                    where = np.searchsorted(start_sites, peak)
                    #print(signalValue, peak, where)
                    if where != 0 and peak <= regions[where-1][1]:
                        addValue = float(signalValue)
                        signal_df.loc[name, chosen_regions[where-1]] = signal_df.loc[name, chosen_regions[where-1]] + addValue
                else:
                    break #after looping all chr1 peaks then stop
    
    # calculate average = signalValue/number of peaks
    average = signal_df.values / data.values
    average = np.nan_to_num(average, 0)
    average = pd.DataFrame(average, index = signal_df.index, columns = signal_df.columns)
    
    return average

In [ ]:
%%time
avg_signalValue_h3k27ac_gene = average_signalValue(h3k27ac_gene_merged, H3K27ac_sorted_path)
#69k columns takes 53 mins

### using pyranges

In [6]:
def average_signalValue_pyranges(data: pd.DataFrame, bedfile_path: str) -> pd.DataFrame:
    #data: pd.DataFrame with region and peak count from each sample, sorted columns accrodding to their position of chr1
    #bedfiles_path: path to the sorted bedfile of sample
    count = 1
    
    #creating signalValue dataframe
    #signal_df = data.copy()
    signal_list = []
       
    #file list in path
    hm_files = os.listdir(bedfile_path)
    
    #string represent of region, region df [chrom, start, end]
    region_df = list_domain_todf(list(data.columns))
    region_df.columns = ['Chromosome', 'Start', 'End']
    region_pr = pr.PyRanges(region_df)
    region_pr.__setattr__('Name', list(data.columns))
    
    
    # getting signalValue
    # sorted file -> chr1 appears first - not necessary  
    for file in hm_files:
        print(1, end='\r', flush=True)
        #sample id
        name = file.split('_')[1].split('.bed')[0]
        
        #read sample in pyranges. get peak positions table
        sample = ReadBED(f"{bedfile_path}/{file}")
        peaks = sample.get_peak_data()
        
        print(2, end='\r', flush=True)
        #extract peak overlap with which region: intersect() -> merge with peak table to get annotate region into matching peak
        peak_overlap_region = region_pr.intersect(peaks).df
        if peak_overlap_region.empty:
            continue
            
        peak_region_match = peaks.df.merge(
            peak_overlap_region[['Chromosome', 'Start', 'End', 'Name']], on=['Chromosome', 'Start', 'End'])  
        
        print(3, end='\r', flush=True)
        #calculate sum of signal value
        sum_signal_value = peak_region_match[['Chromosome', 'signalValue', 'Name']].groupby(['Name']).sum()
        
        print(4, end='\r', flush=True)
        #match signal to regions
        signal_to_region = pd.merge(region_pr.df, sum_signal_value, how='left', on="Name")
        signal_to_region['signalValue'] = signal_to_region['signalValue'].fillna(value=0)
        #print(signal_to_region, flush=True)
        
        print(5, end='\r', flush=True)
        #replace sum signalValue 
        #signal_df.loc[name, :] = signal_to_region['signalValue'].to_numpy()
        signal = signal_to_region['signalValue'].to_numpy()
        avg = signal / data.loc[name, :]
        avg = avg.fillna(value=0)
        row = [name] + list(avg)
        signal_list.append(row)        
        
        print(6, end='\r', flush=True)
        print(f"done with {count} files over {len(hm_files)}", end='\r')
        count += 1 
        
    print(7, end='\r', flush=True)  
    #create dataframe
    fields = ['Sample_ID'] + list(data.columns)
    signal_df = pd.DataFrame(signal_list, columns = fields)
    signal_df = signal_df.set_index('Sample_ID')
  
    print(f"Done processing", end="\r")                
    return signal_df

In [6]:
%%time
pr_signalValue_h3k27ac_gene_ = average_signalValue_pyranges(h3k27ac_gene_merged, H3K27ac_sorted_path)

CPU times: user 4min 12s, sys: 19.1 s, total: 4min 31s
Wall time: 4min 7s


In [7]:
pr_signalValue_h3k27ac_reg = average_signalValue_pyranges(h3k27ac_reg_merged, H3K27ac_sorted_path)
pr_signalValue_h3k27ac_non_gene = average_signalValue_pyranges(h3k27ac_non_gene_merged, H3K27ac_sorted_path)

In [9]:
#save file
write_feather_df(pr_signalValue_h3k27ac_non_gene, '../processing_data/H3K27ac_avgSignalValue_merged_none_gene.feather')
write_feather_df(pr_signalValue_h3k27ac_gene_, '../processing_data/H3K27ac_avgSignalValue_merged_gene.feather')
write_feather_df(pr_signalValue_h3k27ac_reg, '../processing_data/H3K27ac_avgSignalValue_merged_reg.feather')

Done saving dataframe as: ../processing_data/H3K27ac_avgSignalValue_merged_none_gene.feather
Done saving dataframe as: ../processing_data/H3K27ac_avgSignalValue_merged_gene.feather
Done saving dataframe as: ../processing_data/H3K27ac_avgSignalValue_merged_reg.feather


In [11]:
pr_signalValue_h3k27ac_non_gene.head()

,chr1:123686524-123687523,chr1:123719524-123720523,chr1:123802524-123803523,chr1:123892524-123893523,chr1:124367524-124368523,chr1:124410524-124411523,chr1:124720524-124721523,chr1:125007524-125008523,chr1:125064524-125065523,chr1:125066524-125067523,chr1:125084524-125085523,chr1:125161524-125162523
Sample_ID,,,,,,,,,,,,
ENCFF064KHS,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.00000,3.56989,0.0,0.00000,0.00000
ENCFF434KGU,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,4.26821,0.00000,0.0,0.00000,0.00000
ENCFF008FVK,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,4.19806,3.30716
ENCFF115DDQ,0.0,2.12763,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.00000,0.00000
ENCFF051OAR,0.0,0.00000,0.0,0.0,0.0,0.0,7.28885,0.00000,0.00000,0.0,0.00000,0.00000


-------------------------------------

In [4]:
h3k27me3_sorted_path = "../data/sorted_H3K27me3/"
region_type = ['gene', 'reg', 'none_gene']


In [5]:
h3k27me3_gene_merged = read_feather_hm("../processing_data/H3K27me3/H3K27me3_peakcount_merged_gene.feather")
h3k27me3_reg_merged = read_feather_hm("../processing_data/H3K27me3/H3K27me3_peakcount_merged_reg.feather")
h3k27me3_non_gene_merged = read_feather_hm("../processing_data/H3K27me3/H3K27me3_peakcount_merged_none_gene.feather")

In [7]:
for t in region_type:
    if t == 'gene':
        avg_sv = average_signalValue_pyranges(h3k27me3_gene_merged, h3k27me3_sorted_path)
    elif t == 'reg':
        avg_sv = average_signalValue_pyranges(h3k27me3_reg_merged, h3k27me3_sorted_path)
    elif t == 'none_gene':
        avg_sv = average_signalValue_pyranges(h3k27me3_non_gene_merged, h3k27me3_sorted_path)
    write_feather_df(avg_sv, f'../processing_data/H3K27me3/H3K27me3_avgSignalValue_merged_{t}.feather')

Done saving dataframe as: ../processing_data/H3K27me3/H3K27me3_avgSignalValue_merged_gene.feather
Done saving dataframe as: ../processing_data/H3K27me3/H3K27me3_avgSignalValue_merged_reg.feather
Done saving dataframe as: ../processing_data/H3K27me3/H3K27me3_avgSignalValue_merged_none_gene.feather


In [5]:
h3k27me3_exp = pd.read_csv('../Histone_data_tsv/experiment_report_H3K27me3.tsv', delimiter='\t', skiprows=1)
h3k27me3_exp

,ID,Accession,Assay name,Assay title,Biosample classification,Target,Target of assay,Target gene symbol,Biosample summary,Biosample term name,Dbxrefs,Project,Files,Biosample accession,Biological replicate,Technical replicate,Organism,Life stage,Replicates
0,/experiments/ENCSR965BLU/,ENCSR965BLU,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens gastroesophageal sphincter tissue...,gastroesophageal sphincter,GEO:GSE139793,ENCODE,"/files/ENCFF862NQF/,/files/ENCFF425WPG/,/files...",ENCBS727XQL,1,"1,2",Homo sapiens,adult,/replicates/2ba173e8-0aab-46a2-8e30-b8e4169162...
1,/experiments/ENCSR567VAB/,ENCSR567VAB,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens dorsolateral prefrontal cortex ti...,dorsolateral prefrontal cortex,GEO:GSE209056,ENCODE,"/files/ENCFF786FYS/,/files/ENCFF759PXY/,/files...",ENCBS464LBN,1,"1,2",Homo sapiens,adult,/replicates/0dccda4d-b275-4695-87b2-fafef646de...
2,/experiments/ENCSR532EYS/,ENCSR532EYS,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens dorsolateral prefrontal cortex ti...,dorsolateral prefrontal cortex,GEO:GSE208927,ENCODE,"/files/ENCFF754HYY/,/files/ENCFF979WLU/,/files...",ENCBS485UEI,1,"2,1",Homo sapiens,adult,/replicates/4bacc45c-3138-4d06-82fa-5e2361cd68...
3,/experiments/ENCSR558GCO/,ENCSR558GCO,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens dorsolateral prefrontal cortex ti...,dorsolateral prefrontal cortex,GEO:GSE187693,ENCODE,"/files/ENCFF607HEJ/,/files/ENCFF983QXT/,/files...",ENCBS733LER,1,"1,2",Homo sapiens,adult,/replicates/1f1ddc69-6894-4308-8306-0b1a53151e...
4,/experiments/ENCSR469YCE/,ENCSR469YCE,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens upper lobe of left lung tissue ma...,upper lobe of left lung,GEO:GSE139780,ENCODE,"/files/ENCFF128ZRT/,/files/ENCFF602SXE/,/files...",ENCBS943JOF,1,"1,2",Homo sapiens,adult,/replicates/8446d46d-884d-4d12-8f1b-93afd9d66e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,/experiments/ENCSR188HXK/,ENCSR188HXK,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens esophagus muscularis mucosa tissu...,esophagus muscularis mucosa,GEO:GSE139724,ENCODE,"/files/ENCFF161HMR/,/files/ENCFF535WCS/,/files...",ENCBS884JOD,1,"2,1",Homo sapiens,adult,/replicates/2551faef-dac7-40e2-831c-b3720f4c35...
212,/experiments/ENCSR632SLJ/,ENCSR632SLJ,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens Peyer's patch tissue male adult (...,Peyer's patch,GEO:GSE139711,ENCODE,"/files/ENCFF329VRW/,/files/ENCFF268ZAW/,/files...",ENCBS388XLV,1,"1,2",Homo sapiens,adult,/replicates/6bef690b-d6bb-4aea-8329-e4350cc767...
213,/experiments/ENCSR853JJZ/,ENCSR853JJZ,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens chorionic villus tissue female em...,chorionic villus,NaN,Roadmap,"/files/SRR2172735/,/files/SRR2172736/,/files/E...",ENCBS412DCK,1,1,Homo sapiens,embryonic,/replicates/df5f9e4a-f355-463b-9caf-f6b853847f8f/
214,/experiments/ENCSR673XMI/,ENCSR673XMI,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens chorionic villus tissue male embr...,chorionic villus,NaN,Roadmap,"/files/SRR2172763/,/files/SRR2172764/,/files/E...",ENCBS706YBS,1,1,Homo sapiens,embryonic,/replicates/fee6de29-264d-4455-b3e9-741234f4ea57/


In [15]:
h3k27me3_exp.query(f'Files.str.contains("ENCFF964DID")')

,ID,Accession,Assay name,Assay title,Biosample classification,Target,Target of assay,Target gene symbol,Biosample summary,Biosample term name,Dbxrefs,Project,Files,Biosample accession,Biological replicate,Technical replicate,Organism,Life stage,Replicates
11,/experiments/ENCSR453MSI/,ENCSR453MSI,ChIP-seq,Histone ChIP-seq,tissue,/targets/H3K27me3-human/,H3K27me3,"HIST2H3A,HIST2H3D,HIST2H3C",Homo sapiens gastrocnemius medialis tissue fem...,gastrocnemius medialis,GEO:GSE142979,ENCODE,"/files/ENCFF157ZPT/,/files/ENCFF862IWQ/,/files...","ENCBS640TBC,ENCBS731GTN","2,1","1,2",Homo sapiens,adult,/replicates/968a1f1c-3a63-4380-a2f4-8dd3890830...


In [6]:
file_h3k27me3 = os.listdir('../data/H3K27me3/')
infor = {'Code':[],
         'Experiment':[],
         'Target':[],
         'Tissue':[],
         'Biosample_summary':[]}
for f in file_h3k27me3:
    assay_id = f.split('.bed')[0]
    retrieve = h3k27me3_exp.query(f'Files.str.contains("{assay_id}")')
    infor['Code'].append(assay_id)
    infor['Experiment'].append(retrieve['Accession'].item())
    infor['Target'].append(retrieve['Target of assay'].item())
    infor['Tissue'].append(retrieve['Biosample term name'].item())
    infor['Biosample_summary'].append(retrieve['Biosample summary']) 

h3k27me3_info_df = pd.DataFrame.from_dict(infor)

In [7]:
h3k27me3_info_df.head(5)

,Code,Experiment,Target,Tissue,Biosample_summary
0,ENCFF765YZY,ENCSR627STE,H3K27me3,brain,208 Homo sapiens brain tissue male embryo (...
1,ENCFF958KOY,ENCSR042RIW,H3K27me3,sigmoid colon,64 Homo sapiens sigmoid colon tissue male a...
2,ENCFF148VRO,ENCSR177CGN,H3K27me3,esophagus muscularis mucosa,142 Homo sapiens esophagus muscularis mucos...
3,ENCFF608JQT,ENCSR975GDL,H3K27me3,left lung,179 Homo sapiens left lung tissue male adul...
4,ENCFF206GAK,ENCSR611CHC,H3K27me3,right lobe of liver,173 Homo sapiens right lobe of liver tissue...


In [9]:
h3k27me3_info_df.to_csv('../Histone_data_tsv/H3K27me3_tissue_info.csv', index=None)